For CPAIOR 2023

In [87]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [88]:
# import result sets
df_ml = pd.read_csv("Set_1_analysis.csv")
df_ml = df_ml.drop(['GAP W/ CUTS', 'GAP W/O CUTS'], axis = 1)
df_ml.rename(columns={'INITIAL LP':'INITIAL LP W/O CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/O CUTS'}, inplace=True)

df = pd.read_csv("results_1_new.csv")

analysis_ml = df_ml.merge(df, on = ['NAME', 'RANDOM SEED'], how = 'left')

analysis_ml = analysis_ml.drop(['BEST SOLUTION', 'BEST PRIMAL BOUND', 'GAP', 'No. OF CUTS GENERATED', 'No. OF CUTS APPLIED',
                                 'No. OF CUTS PRESENT IN LP AT THE END', 'No. OF NODES', 'RANDOM SEED', 'CUT SETTING',
                                 'STATUS', 'END', 'SOLUTION TIME'], axis = 1)

analysis_ml.rename(columns={'INITIAL LP':'INITIAL LP W/ CUTS', 'PRESOLVING TIME':'PRESOLVING TIME W/ CUTS'}, inplace=True)

analysis_ml['CUTS OFF_FIRST ROUND'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME FIRST ROUND']
analysis_ml['CUTS OFF_ROOT END'] = analysis_ml['TIME W/O CUTS'] + analysis_ml['TIME ROOT END']

analysis_ml = analysis_ml.drop(['TIME ROOT END', 'ROUND OF CUTS', 'OBJECTIVE VALUE ROOT NODE', 'TIME W/O CUTS',
                                'CUTS OFF_ROOT END', 'INITIAL LP TIME'], axis = 1)

analysis_ml = analysis_ml[['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'PRESOLVING TIME W/ CUTS', 'PRESOLVING TIME W/O CUTS',
                            'TIME FIRST ROUND','NODES W/ CUTS', 'NODES W/O CUTS', 'INITIAL LP W/ CUTS', 
                           'INITIAL LP W/O CUTS', 'FIRST ROUND CUT', 'vars', 'bin_vars', 'int_vars', 'cont_vars', 
                           'constr', 'linsetpart_constr', 'linsetpack_constr', 'linsetcov_constr', 'lincard_constr', 
                           'lineqknap_constr', 'linbinpack_constr', 'linknaps_constr', 'linintknap_constr', 'linmixbin_constr',
                           'nonzeroes', 'obj_dyn', 'dynamism_mean', 'presol_vars','presol_int_vars', 'presol_constr']]

analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,11.088517,13.639123,0.067284,0.062201,1.426593,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,11.186637,13.822576,0.057992,0.060087,1.378128,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,11.044666,13.681984,0.055842,0.058780,1.350495,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,10.835616,13.761031,0.052585,0.059116,1.293845,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,10.833424,13.779478,0.053544,0.058801,1.301532,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,wachplan.mps.gz,1427.930347,1804.294142,1.165874,1.552340,2.689512,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1771,wachplan.mps.gz,1444.662760,1807.378575,1.031269,1.914422,2.536143,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1772,wachplan.mps.gz,1431.659624,1739.747262,1.088057,1.507830,2.595248,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1773,wachplan.mps.gz,1433.459125,1761.237671,1.125112,1.547767,2.649715,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [89]:
pd.set_option('display.max_columns', None)

In [90]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,11.088517,13.639123,0.067284,0.062201,1.426593,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
1,10teams.mps.gz,11.186637,13.822576,0.057992,0.060087,1.378128,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
2,10teams.mps.gz,11.044666,13.681984,0.055842,0.058780,1.350495,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
3,10teams.mps.gz,10.835616,13.761031,0.052585,0.059116,1.293845,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
4,10teams.mps.gz,10.833424,13.779478,0.053544,0.058801,1.301532,30,276,20.0,20.0,917.0,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.0,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.0,7.377759,0.000000,5.347108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,wachplan.mps.gz,1427.930347,1804.294142,1.165874,1.552340,2.689512,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1771,wachplan.mps.gz,1444.662760,1807.378575,1.031269,1.914422,2.536143,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1772,wachplan.mps.gz,1431.659624,1739.747262,1.088057,1.507830,2.595248,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025
1773,wachplan.mps.gz,1433.459125,1761.237671,1.125112,1.547767,2.649715,67520,98056,-24.0,-24.0,-9.0,8.119994,0.999702,0.000298,0.000000,7.347944,0.000000,0.000000,0.0,0.144237,0.0,0.0,0.0,0.0,0.018030,0.017120,0.00000,0.0,7.741534,0.000434,6.582025


In [91]:
analysis_ml = analysis_ml.groupby('NAME').mean()
analysis_ml = analysis_ml.reset_index()
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
0,10teams.mps.gz,10.997772,13.736838,0.057449,0.059797,1.350119,30,276,2.000000e+01,2.000000e+01,9.170000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108
1,22433.mps.gz,2.535930,0.823464,0.046082,0.025466,0.172883,1,5,2.124053e+04,2.124053e+04,2.127800e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267
2,23588.mps.gz,5.527370,2.540436,0.027812,0.031946,0.140688,576,981,7.649866e+03,7.649866e+03,7.660000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981
3,30_70_45_05_100.mps.gz,282.653795,258.647469,1.986556,1.625184,18.567687,46,201,-5.336000e+03,-5.336000e+03,8.500000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654
4,30_70_45_095_100.mps.gz,743.997969,75.233190,1.994498,2.143665,19.518728,201,1,-6.132000e+03,-6.132000e+03,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,2446.382658,917.174991,6.005649,5.952476,226.483793,78,11,-2.034000e+00,-2.034000e+00,2.286809e+01,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352
351,umts.mps.gz,1079.840992,11605.745144,0.179507,0.230250,0.892776,144227,3106370,1.460426e+06,1.460426e+06,2.912957e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152
352,unitcal_7.mps.gz,940.799926,10628.686922,6.937303,6.065447,64.953906,705,196989,1.028415e+06,1.028415e+06,1.946934e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349
353,var-smallemery-m6j6.mps.gz,7171.038360,10325.784605,8.873553,8.342061,79.559820,685889,836578,-1.697219e+03,-1.697219e+03,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321


In [92]:
time_label_1 = []

for index, row in analysis_ml.iterrows():
    if row['TIME W/ CUTS'] > row['CUTS OFF_FIRST ROUND']:
        time_label_1.append(-1)
    elif row['TIME W/ CUTS'] <= row['CUTS OFF_FIRST ROUND']:
        time_label_1.append(1)
        
analysis_ml['Time Label'] = time_label_1

In [93]:
analysis_ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,10.997772,13.736838,0.057449,0.059797,1.350119,30,276,2.000000e+01,2.000000e+01,9.170000e+02,7.613325,0.888889,0.000000,0.111111,5.438079,0.347826,0.173913,0.000000,0.000000,0.0,0.0,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,2.535930,0.823464,0.046082,0.025466,0.172883,1,5,2.124053e+04,2.124053e+04,2.127800e+04,6.061457,0.538462,0.000000,0.461538,5.288267,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,-1
2,23588.mps.gz,5.527370,2.540436,0.027812,0.031946,0.140688,576,981,7.649866e+03,7.649866e+03,7.660000e+03,5.908083,0.627717,0.000000,0.372283,4.919981,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,-1
3,30_70_45_05_100.mps.gz,282.653795,258.647469,1.986556,1.625184,18.567687,46,201,-5.336000e+03,-5.336000e+03,8.500000e+00,9.284705,0.999907,0.000000,0.000093,9.396820,0.000000,0.000000,0.004066,0.000000,0.0,0.0,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,-1
4,30_70_45_095_100.mps.gz,743.997969,75.233190,1.994498,2.143665,19.518728,201,1,-6.132000e+03,-6.132000e+03,3.000000e+00,9.303466,0.999909,0.000000,0.000091,9.435562,0.000000,0.000000,0.001437,0.000000,0.0,0.0,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,triptim1.mps.gz,2446.382658,917.174991,6.005649,5.952476,226.483793,78,11,-2.034000e+00,-2.034000e+00,2.286809e+01,10.310784,0.680619,0.319148,0.000233,9.661798,0.000000,0.130460,0.257099,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001092,0.00000,0.002040,10.153818,0.372767,9.661352,-1
351,umts.mps.gz,1079.840992,11605.745144,0.179507,0.230250,0.892776,144227,3106370,1.460426e+06,1.460426e+06,2.912957e+07,7.988543,0.950797,0.024432,0.024771,8.404024,0.033595,0.000000,0.403359,0.002688,0.0,0.0,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
352,unitcal_7.mps.gz,940.799926,10628.686922,6.937303,6.065447,64.953906,705,196989,1.028415e+06,1.028415e+06,1.946934e+07,10.156384,0.110891,0.000000,0.889109,10.798330,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
353,var-smallemery-m6j6.mps.gz,7171.038360,10325.784605,8.873553,8.342061,79.559820,685889,836578,-1.697219e+03,-1.697219e+03,-1.549250e+02,8.631949,0.999643,0.000000,0.000357,9.504203,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [94]:
X = analysis_ml.iloc[0:, 0:-1]
y = analysis_ml.iloc[0:, -1]

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                       'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
cols = X_train.copy()
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
cols

(284, 26) (284,)
(71, 26) (71,)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,bin_vars,int_vars,cont_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linbinpack_constr,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr
298,0.025660,0.042668,0.084510,-9.200187e+03,-9.200187e+03,-9.316389e+02,6.733402,0.057143,0.000000,0.942857,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.114094,0.003428,0.827942,0.234359,6.733402,0.000000,7.083388
247,1.227221,1.474001,41.522818,-2.492000e+03,-2.492000e+03,2.760000e+02,10.386438,0.883395,0.000000,0.116605,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.237443,0.000423,3.000000,0.000000,9.092345,0.000000,9.239025
309,0.029698,0.026762,0.048170,1.324444e+01,1.324444e+01,1.400000e+01,6.133398,0.976139,0.000000,0.023861,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.000000,0.000000,0.661765,0.026668,0.000000,0.758467,5.888878,0.000000,3.970292
336,0.198174,0.307568,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.058960,1.000000,0.000000,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.000000,0.000000,0.244627,0.001267,0.000000,0.213065,7.667626,0.000000,8.767173
142,12.828483,19.519242,35.412962,-2.599800e+04,-2.599800e+04,-2.287477e+04,9.234057,0.975488,0.024512,0.000000,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000105,0.000000,0.000105,0.001384,2.397940,0.054545,9.234057,0.019336,9.158942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2.558282,1.902435,12.338459,4.442776e+08,4.442776e+08,4.444208e+08,9.295967,1.000000,0.000000,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.000000,0.000000,0.219394,0.035195,1.531250,0.098539,9.295967,0.000000,6.715383
192,11.176662,15.067371,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,10.619814,0.127656,0.000000,0.872344,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000108,0.000054,0.000000,0.350567,0.000121,2.595280,0.018692,10.619814,0.000000,11.617520
117,0.880457,1.154996,1.493279,3.000000e+00,3.000000e+00,3.000000e+00,8.974745,0.970130,0.029870,0.000000,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.094330,0.000000,0.000737,0.000000,2.724850,8.023225,0.037365,8.583917
47,0.247610,0.309220,6.985466,3.000000e+00,3.000000e+00,3.000000e+00,9.128696,1.000000,0.000000,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000434,0.000000,0.000000,9.128696,0.000000,9.041567


In [96]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [97]:
cols = cols.iloc[0:, 0:]

bestfeatures = SelectKBest(score_func=f_classif, k = 'all')
fit = bestfeatures.fit(X_train,y_train)
scores = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(cols.columns)
featureScores = pd.concat([columns,scores],axis=1)
featureScores
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(26,'Score')) #print best features
data=featureScores.nlargest(26,'Score')

                    Features     Score
19          linmixbin_constr  6.298618
8                   int_vars  3.648725
13          linsetcov_constr  3.634335
20                 nonzeroes  2.765360
24           presol_int_vars  2.036928
14            lincard_constr  1.644193
12         linsetpack_constr  1.596735
3         INITIAL LP W/ CUTS  1.414680
4        INITIAL LP W/O CUTS  1.414680
15          lineqknap_constr  0.752474
22             dynamism_mean  0.742226
5            FIRST ROUND CUT  0.679703
2           TIME FIRST ROUND  0.638720
21                   obj_dyn  0.523242
10                    constr  0.458131
7                   bin_vars  0.420117
11         linsetpart_constr  0.395539
25             presol_constr  0.353088
18         linintknap_constr  0.294091
16         linbinpack_constr  0.255127
17           linknaps_constr  0.252387
0    PRESOLVING TIME W/ CUTS  0.229901
9                  cont_vars  0.145885
23               presol_vars  0.107399
6                       v

In [98]:
X = analysis_ml.drop(['PRESOLVING TIME W/O CUTS', 'vars', 'presol_vars', 'cont_vars',
                      'PRESOLVING TIME W/ CUTS', 'linknaps_constr', 'linbinpack_constr', 'Time Label'], axis = 1)
y = analysis_ml.iloc[0:, -1]

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

test_df = X_test.copy()
train_df = X_train.copy()

X_train = X_train.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 
                        'NODES W/O CUTS'], axis = 1)
X_test = X_test.drop(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND',
                     'NODES W/ CUTS', 'NODES W/O CUTS'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(284, 19) (284,)
(71, 19) (71,)


,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,bin_vars,int_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_int_vars,presol_constr
298,0.084510,-9.200187e+03,-9.200187e+03,-9.316389e+02,0.057143,0.000000,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.114094,0.003428,0.827942,0.234359,0.000000,7.083388
247,41.522818,-2.492000e+03,-2.492000e+03,2.760000e+02,0.883395,0.000000,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.237443,0.000423,3.000000,0.000000,0.000000,9.239025
309,0.048170,1.324444e+01,1.324444e+01,1.400000e+01,0.976139,0.000000,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.661765,0.026668,0.000000,0.758467,0.000000,3.970292
336,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.244627,0.001267,0.000000,0.213065,0.000000,8.767173
142,35.412962,-2.599800e+04,-2.599800e+04,-2.287477e+04,0.975488,0.024512,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000105,0.001384,2.397940,0.054545,0.019336,9.158942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,12.338459,4.442776e+08,4.442776e+08,4.444208e+08,1.000000,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.219394,0.035195,1.531250,0.098539,0.000000,6.715383
192,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.127656,0.000000,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000000,0.350567,0.000121,2.595280,0.018692,0.000000,11.617520
117,1.493279,3.000000e+00,3.000000e+00,3.000000e+00,0.970130,0.029870,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.094330,0.000000,0.000737,0.000000,2.724850,0.037365,8.583917
47,6.985466,3.000000e+00,3.000000e+00,3.000000e+00,1.000000,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000434,0.000000,0.000000,0.000000,9.041567


In [100]:
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [101]:
model_params={'svm':{'model': SVC(kernel = 'rbf', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 'gamma': [1e-2, 1e-3, 1e-4]
                             }
                    },
              'rfm':{'model': RandomForestClassifier(random_state = 0), 
                    'params':{'n_estimators':[100, 200, 500], 
                             'max_depth':[5, 10, 15]
                             }
                    },
              'lrm':{'model': LogisticRegression(solver='lbfgs', random_state = 0), 
                    'params':{'C':[10, 100, 1000], 
                              'max_iter':[100, 500, 1000]
                             }
                    },
              'et':{'model': ExtraTreesClassifier(random_state=0), 
                    'params':{'n_estimators': [100, 200, 500], 
                              'max_depth': [5, 10, 15]
                             }
                    },
              'nn':{'model': MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', 
                                           power_t=0.5, max_iter=500, shuffle=True, tol=0.00001, verbose=True, 
                                           early_stopping=False, validation_fraction=0.1, random_state=0),
                    'params':{'hidden_layer_sizes': [(5, ), (10, ), (15, )],
                                 'learning_rate_init': [1e-1, 1e-2, 1e-3],
                                 'alpha':  [1e-2, 1e-3, 1e-4],
                                 'activation': ['tanh', 'relu', 'logistic']
                             }
                    }
             }
scores=[]
for model_name, mp in model_params.items():
    clf=GridSearchCV(mp['model'], mp['params'], cv = None, return_train_score = False, n_jobs = -1)
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score':clf.best_score_, 'best_params':clf.best_params_})

C:\Users\ambar\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [126]:
scores=pd.DataFrame(scores).sort_values(by = ['best_score'], ascending = False)
scores.to_csv('set_1_scores_new_1.csv')

PermissionError: [Errno 13] Permission denied: 'set_1_scores_new_1.csv'

In [135]:
scores = OrderedDict()
target_names = ['-1', '1']

print('SUPPORT VECTOR MACHINES')
clf = SVC(random_state = 0, C = 100, gamma = 0.01, kernel = 'rbf')
clf.fit(X_train, y_train)
y_train_pred_svm = clf.predict(X_train)
y_test_pred_svm = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_svm)
mse_test = mean_squared_error(y_test, y_test_pred_svm)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_svm, target_names=target_names))
scores['SVC'] = [score, metrics.precision_score(y_test, y_test_pred_svm), metrics.recall_score(y_test, y_test_pred_svm),
                      metrics.f1_score(y_test, y_test_pred_svm), mse_test, mse_train]

print('##################################')
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(15, ), activation='logistic', alpha=0.01, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_nn)
mse_test = mean_squared_error(y_test, y_test_pred_nn)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
                      metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]

print('##################################')
print('DECISION TREES')
clf = ExtraTreesClassifier(random_state=0, max_depth = 15, n_estimators = 200)
clf.fit(X_train, y_train)
y_train_pred_et = clf.predict(X_train)
y_test_pred_et = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_et)
mse_test = mean_squared_error(y_test, y_test_pred_et)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_et, target_names=target_names))
scores['ET'] = [score, metrics.precision_score(y_test, y_test_pred_et), 
                         metrics.recall_score(y_test, y_test_pred_et), 
                         metrics.f1_score(y_test, y_test_pred_et), mse_test, mse_train]

print('##################################')
print('LOGISTIC REGRESSION')
clf = LogisticRegression(solver='lbfgs', random_state = 0, C = 100, max_iter = 100)
clf.fit(X_train, y_train)
y_train_pred_lr = clf.predict(X_train)
y_test_pred_lr = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_lr)
mse_test = mean_squared_error(y_test, y_test_pred_lr)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_lr, target_names=target_names))
scores['LR'] = [score, metrics.precision_score(y_test, y_test_pred_lr), metrics.recall_score(y_test, y_test_pred_lr),
                      metrics.f1_score(y_test, y_test_pred_lr), mse_test, mse_train]

print('##################################')
print('RANDOM FOREST')
clf = RandomForestClassifier(random_state = 0, max_depth = 5, n_estimators = 100)
clf.fit(X_train, y_train)
y_train_pred_rf = clf.predict(X_train)
y_test_pred_rf = clf.predict(X_test)
score = clf.score(X_test, y_test)
mse_train = mean_squared_error(y_train, y_train_pred_rf)
mse_test = mean_squared_error(y_test, y_test_pred_rf)
print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score: {}\n".format(score))
print(classification_report(y_test, y_test_pred_rf, target_names=target_names))
scores['RF'] = [score, metrics.precision_score(y_test, y_test_pred_rf), metrics.recall_score(y_test, y_test_pred_rf),
                      metrics.f1_score(y_test, y_test_pred_rf), mse_test, mse_train]

SUPPORT VECTOR MACHINES
MSE Test: 2.1971830985915495, MSE Train: 1.0
Accuracy score: 0.4507042253521127

              precision    recall  f1-score   support

          -1       0.28      0.16      0.20        31
           1       0.51      0.68      0.58        40

    accuracy                           0.45        71
   macro avg       0.39      0.42      0.39        71
weighted avg       0.41      0.45      0.42        71

##################################
NEURAL NETWORKS


C:\Users\ambar\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MSE Test: 2.028169014084507, MSE Train: 0.04225352112676056
Accuracy score: 0.49295774647887325

              precision    recall  f1-score   support

          -1       0.42      0.42      0.42        31
           1       0.55      0.55      0.55        40

    accuracy                           0.49        71
   macro avg       0.48      0.48      0.48        71
weighted avg       0.49      0.49      0.49        71

##################################
DECISION TREES
MSE Test: 1.971830985915493, MSE Train: 0.07042253521126761
Accuracy score: 0.5070422535211268

              precision    recall  f1-score   support

          -1       0.39      0.23      0.29        31
           1       0.55      0.72      0.62        40

    accuracy                           0.51        71
   macro avg       0.47      0.48      0.45        71
weighted avg       0.48      0.51      0.48        71

##################################
LOGISTIC REGRESSION
MSE Test: 1.6901408450704225, MSE Train: 1.39436

In [136]:
scores_df = pd.DataFrame.from_dict(scores, orient='index')
scores_df.columns=['accuracy', 'precision', 'recall', 'f1-score', 'mse test', 'mse train']
scores_df

,accuracy,precision,recall,f1-score,mse test,mse train
SVC,0.450704,0.509434,0.675,0.580645,2.197183,1.000000
NN,0.492958,0.550000,0.550,0.550000,2.028169,0.042254
ET,0.507042,0.547170,0.725,0.623656,1.971831,0.070423
LR,0.577465,0.592593,0.800,0.680851,1.690141,1.394366
RF,0.605634,0.600000,0.900,0.720000,1.577465,0.492958


In [137]:
test_df['PLabel NN'] = y_test_pred_nn
test_df['PLabel ET'] = y_test_pred_et
test_df['PLabel RF'] = y_test_pred_rf

test_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = test_df.merge(test_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_FIRST ROUND', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,30n20b8.mps.gz,152.430964,167.194716,1617,13632,-1,1,1,1,152.430964,1617
1,mspsphard01i.mps.gz,10493.451548,3456.376067,323371,152457,1,1,1,-1,3456.376067,152457
2,rail507.mps.gz,183.736345,213.916191,836,1010,1,1,1,1,183.736345,836
3,neos-5193246-nerang.mps.gz,1171.813190,1544.746675,7082,17117,1,1,1,1,1171.813190,7082
4,gen-ip002.mps.gz,1482.181497,1182.168532,5859548,5112787,1,1,1,-1,1182.168532,5112787
...,...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,18.061062,21,29199,1,1,1,1,1.927490,21
67,neos-1601936.mps.gz,11527.267046,14964.550487,14529,1,-1,-1,1,1,11527.267046,1
68,neos-1425699.mps.gz,0.011907,0.029142,1,1,1,1,1,1,0.011907,1
69,neos-4322846-ryton.mps.gz,818.066564,845.596316,664,522,-1,1,1,1,818.066564,522


In [138]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,30n20b8.mps.gz,152.430964,167.194716,1617,13632,-1,1,1,1,152.430964,1617,152.430964,167.194716,152.430964,1617,13632,1617
1,mspsphard01i.mps.gz,10493.451548,3456.376067,323371,152457,1,1,1,-1,3456.376067,152457,10493.451548,10493.451548,10493.451548,323371,323371,323371
2,rail507.mps.gz,183.736345,213.916191,836,1010,1,1,1,1,183.736345,836,183.736345,183.736345,183.736345,836,836,836
3,neos-5193246-nerang.mps.gz,1171.813190,1544.746675,7082,17117,1,1,1,1,1171.813190,7082,1171.813190,1171.813190,1171.813190,7082,7082,7082
4,gen-ip002.mps.gz,1482.181497,1182.168532,5859548,5112787,1,1,1,-1,1182.168532,5112787,1482.181497,1482.181497,1482.181497,5859548,5859548,5859548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,neos-3611689-kaihu.mps.gz,1.927490,18.061062,21,29199,1,1,1,1,1.927490,21,1.927490,1.927490,1.927490,21,21,21
67,neos-1601936.mps.gz,11527.267046,14964.550487,14529,1,-1,-1,1,1,11527.267046,1,11527.267046,14964.550487,14964.550487,14529,1,1
68,neos-1425699.mps.gz,0.011907,0.029142,1,1,1,1,1,1,0.011907,1,0.011907,0.011907,0.011907,1,1,1
69,neos-4322846-ryton.mps.gz,818.066564,845.596316,664,522,-1,1,1,1,818.066564,522,818.066564,845.596316,818.066564,664,522,664


In [139]:
testing = analysis.copy()

In [140]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [141]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [142]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [143]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [144]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      274.255724    304.319538   210.563040   
               [10,18000)     526.420781    561.655930   383.920418   
               [100,18000)   1308.766320   1332.380249   957.653597   
               [1000,18000)  4338.893940   4090.017348  2778.446249   
               [3600,18000)  7878.133243  10499.550292  6283.397169   

Parameter                                                            \
Setting                               RF           ET            NN   
Metric         Bucket                                                 
Geometric Mean [0,18000)      256.560120   275.755010    269.263330   
               [10,18000)     485.210127   530.663871    515.779240   
               [100,18000)   1207.681406  1411.746478   1411.643062   
               [1000,18000)  3559.845106  4687.616610   4429.137472   
               [3600,18000)  7557.597746  9974.853795  10499.550292   

Parameter                             Node                               \
Setting                                Cut         No Cut       Perfect   
Metric         Bucket                                                     
Geometric Mean [0,18000)       5873.772645   12660.610956   5074.993594   
               [10,18000)     11162.501804   23205.981020   9343.101918   
               [100,18000)    24389.263707   44079.858893  19612.674466   
               [1000,18000)  131214.637044  290463.280989  93209.687204   
               [3600,18000)  127719.553802  320029.782193  80506.633729   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6111.645572    6324.852839    7389.587221   
               [10,18000)     11729.629481   12129.763098   14840.431585   
               [100,18000)    25942.568497   27547.692415   38170.386852   
               [1000,18000)  138717.226103  149855.699875  237011.359567   
               [3600,18000)  133200.355963  126517.456193  320029.782193   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)           71  
               [10,18000)          53  
               [100,18000)         34  
               [1000,18000)        16  
               [3600,18000)         6

In [145]:
imp_time_nn = []
imp_nodes_nn = []

for i, j in zip(t_nn, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_nn.append(imp)

for i, j in zip(n_nn, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_nn.append(imp)

In [146]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_nn
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_nn
avg_bb_df

Parameter                           Time                             \
Setting                              Cut        No Cut      Perfect   
Metric         Bucket                                                 
Geometric Mean [0,18000)      274.255724    304.319538   210.563040   
               [10,18000)     526.420781    561.655930   383.920418   
               [100,18000)   1308.766320   1332.380249   957.653597   
               [1000,18000)  4338.893940   4090.017348  2778.446249   
               [3600,18000)  7878.133243  10499.550292  6283.397169   

Parameter                                                            \
Setting                               RF           ET            NN   
Metric         Bucket                                                 
Geometric Mean [0,18000)      256.560120   275.755010    269.263330   
               [10,18000)     485.210127   530.663871    515.779240   
               [100,18000)   1207.681406  1411.746478   1411.643062   
               [1000,18000)  3559.845106  4687.616610   4429.137472   
               [3600,18000)  7557.597746  9974.853795  10499.550292   

Parameter                             Node                               \
Setting                                Cut         No Cut       Perfect   
Metric         Bucket                                                     
Geometric Mean [0,18000)       5873.772645   12660.610956   5074.993594   
               [10,18000)     11162.501804   23205.981020   9343.101918   
               [100,18000)    24389.263707   44079.858893  19612.674466   
               [1000,18000)  131214.637044  290463.280989  93209.687204   
               [3600,18000)  127719.553802  320029.782193  80506.633729   

Parameter                                                                 \
Setting                                 RF             ET             NN   
Metric         Bucket                                                      
Geometric Mean [0,18000)       6111.645572    6324.852839    7389.587221   
               [10,18000)     11729.629481   12129.763098   14840.431585   
               [100,18000)    25942.568497   27547.692415   38170.386852   
               [1000,18000)  138717.226103  149855.699875  237011.359567   
               [3600,18000)  133200.355963  126517.456193  320029.782193   

Parameter                   Instances Improvement (%)              
Setting                                          Time       Nodes  
Metric         Bucket                                              
Geometric Mean [0,18000)           71       -1.820343   25.806490  
               [10,18000)          53       -2.021490   32.948974  
               [100,18000)         34        7.860589   56.504876  
               [1000,18000)        16        2.079874   80.628751  
               [3600,18000)         6       33.274597  150.572268

In [147]:
train_df

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,bin_vars,int_vars,constr,linsetpart_constr,linsetpack_constr,linsetcov_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_int_vars,presol_constr,PLabel NN,PLabel ET,PLabel RF
298,qiu.mps.gz,68.293240,42.959414,0.084510,10652,9207,-9.200187e+03,-9.200187e+03,-9.316389e+02,0.057143,0.000000,7.083388,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.114094,0.003428,0.827942,0.234359,0.000000,7.083388,-1,-1,-1
247,neos-933638.mps.gz,14587.569504,1330.715756,41.522818,983,64,-2.492000e+03,-2.492000e+03,2.760000e+02,0.883395,0.000000,9.522081,0.000000,0.392224,0.000000,0.000000,0.0,0.000000,0.237443,0.000423,3.000000,0.000000,0.000000,9.239025,-1,-1,-1
309,rlp1.mps.gz,0.211836,0.264443,0.048170,51,225,1.324444e+01,1.324444e+01,1.400000e+01,0.976139,0.000000,4.219508,0.000000,0.000000,0.000000,0.147059,0.0,0.000000,0.661765,0.026668,0.000000,0.758467,0.000000,3.970292,1,1,1
336,supportcase4.mps.gz,3.806084,113.729586,0.000000,1,160,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,0.000000,9.158205,0.000000,0.000000,0.647282,0.000000,0.0,0.000000,0.244627,0.001267,0.000000,0.213065,0.000000,8.767173,1,1,1
142,mzzv11.mps.gz,422.340278,1525.322799,35.412962,1356,8880,-2.599800e+04,-2.599800e+04,-2.287477e+04,0.975488,0.024512,9.158942,0.112959,0.605853,0.002948,0.232446,0.0,0.000000,0.000105,0.001384,2.397940,0.054545,0.019336,9.158942,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,sp98ic.mps.gz,8603.418640,13271.468537,12.338459,141964,432282,4.442776e+08,4.442776e+08,4.444208e+08,1.000000,0.000000,6.715383,0.000000,0.756364,0.020606,0.000000,0.0,0.000000,0.219394,0.035195,1.531250,0.098539,0.000000,6.715383,1,1,1
192,neos-3755335-nizao.mps.gz,16.354954,16.846205,0.000000,1,1,0.000000e+00,0.000000e+00,0.000000e+00,0.127656,0.000000,11.617520,0.002072,0.000486,0.000000,0.000000,0.0,0.000000,0.350567,0.000121,2.595280,0.018692,0.000000,11.617520,-1,1,1
117,lectsched-4-obj.mps.gz,5.974980,19.359423,1.493279,5,1395,3.000000e+00,3.000000e+00,3.000000e+00,0.970130,0.029870,9.558388,0.000000,0.000000,0.000000,0.000000,0.0,0.094330,0.000000,0.000737,0.000000,2.724850,0.037365,8.583917,1,1,1
47,chromaticindex128-5.mps.gz,84.350632,1199.593151,6.985466,1316,100314,3.000000e+00,3.000000e+00,3.000000e+00,1.000000,0.000000,9.041567,0.272641,0.727359,0.000000,0.000000,0.0,0.000000,0.000000,0.000434,0.000000,0.000000,0.000000,9.041567,1,1,1


In [148]:
train_df['PLabel NN'] = y_train_pred_nn
train_df['PLabel ET'] = y_train_pred_et
train_df['PLabel RF'] = y_train_pred_rf
train_feat = analysis_ml.filter(['NAME', 'Time Label'])
analysis = train_df.merge(train_feat, on = ['NAME'], how = 'left')
analysis = analysis.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 
                            'NODES W/ CUTS', 'NODES W/O CUTS', 'PLabel NN', 'PLabel ET', 'PLabel RF', 'Time Label'])
analysis['Minimum Time'] = analysis[['CUTS OFF_FIRST ROUND', 'TIME W/ CUTS']].min(axis = 1)
analysis['Minimum Nodes'] = analysis[['NODES W/O CUTS', 'NODES W/ CUTS']].min(axis = 1)

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes
0,qiu.mps.gz,68.293240,42.959414,10652,9207,-1,-1,1,-1,42.959414,9207
1,neos-933638.mps.gz,14587.569504,1330.715756,983,64,-1,-1,-1,-1,1330.715756,64
2,rlp1.mps.gz,0.211836,0.264443,51,225,1,1,1,1,0.211836,51
3,supportcase4.mps.gz,3.806084,113.729586,1,160,1,1,-1,1,3.806084,1
4,mzzv11.mps.gz,422.340278,1525.322799,1356,8880,1,1,1,1,422.340278,1356
...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13271.468537,141964,432282,1,1,1,1,8603.418640,141964
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1,1,1,1,1,1,16.354954,1
281,lectsched-4-obj.mps.gz,5.974980,19.359423,5,1395,1,1,1,1,5.974980,5
282,chromaticindex128-5.mps.gz,84.350632,1199.593151,1316,100314,1,1,1,1,84.350632,1316


In [149]:
rf_pred_time = []
rf_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel RF'] == 1.0:
        rf_pred_time.append(row['TIME W/ CUTS'])
        rf_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel RF'] == -1.0:
        rf_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        rf_pred_nodes.append(row['NODES W/O CUTS'])

nn_pred_time = []
nn_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel NN'] == 1.0:
        nn_pred_time.append(row['TIME W/ CUTS'])
        nn_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel NN'] == -1.0:
        nn_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        nn_pred_nodes.append(row['NODES W/O CUTS'])

et_pred_time = []
et_pred_nodes = []        
for index, row in analysis.iterrows():
    if row['PLabel ET'] == 1.0:
        et_pred_time.append(row['TIME W/ CUTS'])
        et_pred_nodes.append(row['NODES W/ CUTS'])
    elif row['PLabel ET'] == -1.0:
        et_pred_time.append(row['CUTS OFF_FIRST ROUND'])
        et_pred_nodes.append(row['NODES W/O CUTS'])

        
analysis['PTime RF'] = rf_pred_time
analysis['PTime NN'] = nn_pred_time
analysis['PTime ET'] = et_pred_time

analysis['PNodes RF'] = rf_pred_nodes
analysis['PNodes NN'] = nn_pred_nodes
analysis['PNodes ET'] = et_pred_nodes

analysis

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,PLabel NN,PLabel ET,PLabel RF,Time Label,Minimum Time,Minimum Nodes,PTime RF,PTime NN,PTime ET,PNodes RF,PNodes NN,PNodes ET
0,qiu.mps.gz,68.293240,42.959414,10652,9207,-1,-1,1,-1,42.959414,9207,68.293240,42.959414,42.959414,10652,9207,9207
1,neos-933638.mps.gz,14587.569504,1330.715756,983,64,-1,-1,-1,-1,1330.715756,64,1330.715756,1330.715756,1330.715756,64,64,64
2,rlp1.mps.gz,0.211836,0.264443,51,225,1,1,1,1,0.211836,51,0.211836,0.211836,0.211836,51,51,51
3,supportcase4.mps.gz,3.806084,113.729586,1,160,1,1,-1,1,3.806084,1,113.729586,3.806084,3.806084,160,1,1
4,mzzv11.mps.gz,422.340278,1525.322799,1356,8880,1,1,1,1,422.340278,1356,422.340278,422.340278,422.340278,1356,1356,1356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,sp98ic.mps.gz,8603.418640,13271.468537,141964,432282,1,1,1,1,8603.418640,141964,8603.418640,8603.418640,8603.418640,141964,141964,141964
280,neos-3755335-nizao.mps.gz,16.354954,16.846205,1,1,1,1,1,1,16.354954,1,16.354954,16.354954,16.354954,1,1,1
281,lectsched-4-obj.mps.gz,5.974980,19.359423,5,1395,1,1,1,1,5.974980,5,5.974980,5.974980,5.974980,5,5,5
282,chromaticindex128-5.mps.gz,84.350632,1199.593151,1316,100314,1,1,1,1,84.350632,1316,84.350632,84.350632,84.350632,1316,1316,1316


In [150]:
training = analysis.copy()

In [151]:
bb_classes = ['Geometric Mean']
num_bb_classes = len(bb_classes)

bucket_min = [0, 10, 100, 1000, 3600]
bucket_max = [18000, 18000, 18000, 18000, 18000]
num_buckets = len(bucket_min)
assert(len(bucket_max) == num_buckets)
bb_buckets = ['[' + str(bucket_min[j]) + ',' + str(bucket_max[j]) + ')' for j in range(num_buckets)]

bb_metrics = ['Gmean']

In [152]:
avg_bb_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut','No Cut', 'Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

bb_row_names = pd.MultiIndex.from_product(
    [bb_classes, bb_buckets],
    names=['Metric', 'Bucket'])

avg_bb_df = pd.DataFrame(
    index = bb_row_names,
    dtype = float)

avg_bb_df = pd.DataFrame(
    columns = avg_bb_cols,
    index = bb_row_names,
    dtype = float)

In [153]:
df_0 = analysis[(analysis['Minimum Time'] >= 0)]
df_10 = analysis[(analysis['Minimum Time'] >= 10)]
df_100 = analysis[(analysis['Minimum Time'] >= 100)]
df_1000 = analysis[(analysis['Minimum Time'] >= 1000)]
df_3600 = analysis[(analysis['Minimum Time'] >= 3600)]

In [154]:
SHIFT_TIME = 60
SHIFT_NODES = 1000

df = [df_0, df_10, df_100, df_1000, df_3600]
t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [155]:
avg_bb_df['Instances'] = instances

avg_bb_df[('Time', 'Cut')] = t_cuts
avg_bb_df[('Time', 'No Cut')] = t_nocuts

avg_bb_df[('Time', 'RF')] = t_rf
avg_bb_df[('Time', 'ET')] = t_et
avg_bb_df[('Time', 'NN')] = t_nn

avg_bb_df[('Time', 'Perfect')] = t_perfect

avg_bb_df[('Node', 'Cut')] = n_cuts
avg_bb_df[('Node', 'No Cut')] = n_nocuts

avg_bb_df[('Node', 'RF')] = n_rf
avg_bb_df[('Node', 'ET')] = n_et
avg_bb_df[('Node', 'NN')] = n_nn

avg_bb_df[('Node', 'Perfect')] = n_perfect

avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   247.106544   143.960513   
               [10,18000)     318.077008   422.335487   244.689919   
               [100,18000)    801.945867   999.958655   601.956774   
               [1000,18000)  3197.953661  3796.393283  2324.242373   
               [3600,18000)  7998.252381  9147.050248  7321.486897   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      153.385419   144.681741   144.370924   
               [10,18000)     262.060961   246.134121   245.511522   
               [100,18000)    635.373148   606.595181   603.711693   
               [1000,18000)  2517.443549  2360.654590  2332.853090   
               [3600,18000)  7321.486897  7321.486897  7321.486897   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2610.230290    7071.275598   2308.375949   
               [10,18000)     4381.790018   11259.477754   3788.838502   
               [100,18000)    7980.565100   17096.106110   6854.929574   
               [1000,18000)  30745.310546   57742.655258  23559.661294   
               [3600,18000)  71905.735126  131109.440239  68851.825194   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2591.150202   2561.117038   2561.288414   
               [10,18000)     4312.825554   4252.421442   4252.760059   
               [100,18000)    7779.858659   7871.768184   7857.671563   
               [1000,18000)  27397.082080  25690.053359  25541.528177   
               [3600,18000)  84513.407387  84513.407387  84513.407387   

Parameter                   Instances  
Setting                                
Metric         Bucket                  
Geometric Mean [0,18000)          284  
               [10,18000)         212  
               [100,18000)        127  
               [1000,18000)        39  
               [3600,18000)         8

In [156]:
imp_time_nn = []
imp_nodes_nn = []

for i, j in zip(t_nn, t_cuts):
    imp = ((i-j)/j)*100
    imp_time_nn.append(imp)

for i, j in zip(n_nn, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes_nn.append(imp)

In [157]:
avg_bb_df[('Improvement (%)', 'Time')] = imp_time_nn
avg_bb_df[('Improvement (%)', 'Nodes')] = imp_nodes_nn
avg_bb_df

Parameter                           Time                            \
Setting                              Cut       No Cut      Perfect   
Metric         Bucket                                                
Geometric Mean [0,18000)      180.289030   247.106544   143.960513   
               [10,18000)     318.077008   422.335487   244.689919   
               [100,18000)    801.945867   999.958655   601.956774   
               [1000,18000)  3197.953661  3796.393283  2324.242373   
               [3600,18000)  7998.252381  9147.050248  7321.486897   

Parameter                                                           \
Setting                               RF           ET           NN   
Metric         Bucket                                                
Geometric Mean [0,18000)      153.385419   144.681741   144.370924   
               [10,18000)     262.060961   246.134121   245.511522   
               [100,18000)    635.373148   606.595181   603.711693   
               [1000,18000)  2517.443549  2360.654590  2332.853090   
               [3600,18000)  7321.486897  7321.486897  7321.486897   

Parameter                            Node                               \
Setting                               Cut         No Cut       Perfect   
Metric         Bucket                                                    
Geometric Mean [0,18000)      2610.230290    7071.275598   2308.375949   
               [10,18000)     4381.790018   11259.477754   3788.838502   
               [100,18000)    7980.565100   17096.106110   6854.929574   
               [1000,18000)  30745.310546   57742.655258  23559.661294   
               [3600,18000)  71905.735126  131109.440239  68851.825194   

Parameter                                                              \
Setting                                RF            ET            NN   
Metric         Bucket                                                   
Geometric Mean [0,18000)      2591.150202   2561.117038   2561.288414   
               [10,18000)     4312.825554   4252.421442   4252.760059   
               [100,18000)    7779.858659   7871.768184   7857.671563   
               [1000,18000)  27397.082080  25690.053359  25541.528177   
               [3600,18000)  84513.407387  84513.407387  84513.407387   

Parameter                   Instances Improvement (%)             
Setting                                          Time      Nodes  
Metric         Bucket                                             
Geometric Mean [0,18000)          284      -19.922514  -1.875002  
               [10,18000)         212      -22.813811  -2.944686  
               [100,18000)        127      -24.719147  -1.539910  
               [1000,18000)        39      -27.051692 -16.925451  
               [3600,18000)         8       -8.461417  17.533612

In [158]:
df = [training, testing]

t_cuts = []
t_nocuts = []
t_perfect = []
t_rf = []
t_et = []
t_nn = []

n_cuts = []
n_nocuts = []
n_perfect = []
n_rf = []
n_et = []
n_nn = []

instances = []

for data in df:
    time_nocuts = data['CUTS OFF_FIRST ROUND'].tolist()
    time_cuts = data['TIME W/ CUTS'].tolist()
    time_perfect = data['Minimum Time'].tolist()
    time_rf = data['PTime RF'].tolist()
    time_et = data['PTime ET'].tolist()
    time_nn = data['PTime NN'].tolist()
    
    timeshift_nocuts = [i + SHIFT_TIME for i in time_nocuts]
    timeshift_cuts = [i + SHIFT_TIME for i in time_cuts]
    timeshift_perfect = [i + SHIFT_TIME for i in time_perfect]
    timeshift_rf = [i + SHIFT_TIME for i in time_rf]
    timeshift_et = [i + SHIFT_TIME for i in time_et]
    timeshift_nn = [i + SHIFT_TIME for i in time_nn]
    
    
    time_geom_nocuts = geometric_mean(timeshift_nocuts) - SHIFT_TIME
    time_geom_cuts = geometric_mean(timeshift_cuts) - SHIFT_TIME
    time_geom_perfect = geometric_mean(timeshift_perfect) - SHIFT_TIME
    time_geom_rf = geometric_mean(timeshift_rf) - SHIFT_TIME
    time_geom_et = geometric_mean(timeshift_et) - SHIFT_TIME
    time_geom_nn = geometric_mean(timeshift_nn) - SHIFT_TIME
    
    
    t_nocuts.append(time_geom_nocuts)
    t_cuts.append(time_geom_cuts)
    t_perfect.append(time_geom_perfect)
    t_rf.append(time_geom_rf)
    t_et.append(time_geom_et)
    t_nn.append(time_geom_nn)
    
    nodes_nocuts = data['NODES W/O CUTS'].tolist()
    nodes_cuts = data['NODES W/ CUTS'].tolist()
    nodes_perfect = data['Minimum Nodes'].tolist()
    nodes_rf = data['PNodes RF'].tolist()
    nodes_et = data['PNodes ET'].tolist()
    nodes_nn = data['PNodes NN'].tolist()
    
    nodeshift_nocuts = [i + SHIFT_NODES for i in nodes_nocuts]
    nodeshift_cuts = [i + SHIFT_NODES for i in nodes_cuts]
    nodeshift_perfect = [i + SHIFT_NODES for i in nodes_perfect]
    nodeshift_rf = [i + SHIFT_NODES for i in nodes_rf]
    nodeshift_et = [i + SHIFT_NODES for i in nodes_et]
    nodeshift_nn = [i + SHIFT_NODES for i in nodes_nn]
    
    nodes_geom_nocuts = geometric_mean(nodeshift_nocuts) - SHIFT_NODES
    nodes_geom_cuts = geometric_mean(nodeshift_cuts) - SHIFT_NODES
    nodes_geom_perfect = geometric_mean(nodeshift_perfect) - SHIFT_NODES
    nodes_geom_rf = geometric_mean(nodeshift_rf) - SHIFT_NODES
    nodes_geom_et = geometric_mean(nodeshift_et) - SHIFT_NODES
    nodes_geom_nn = geometric_mean(nodeshift_nn) - SHIFT_NODES
    
    n_nocuts.append(nodes_geom_nocuts)
    n_cuts.append(nodes_geom_cuts)
    n_perfect.append(nodes_geom_perfect)
    n_rf.append(nodes_geom_rf)
    n_et.append(nodes_geom_et)
    n_nn.append(nodes_geom_nn)
    
    instances.append(len(data))

In [159]:
train_test_classes = ['Geometric Mean']
train_test_sets = ['Train Set', 'Test Set']

In [160]:
train_test_cols = pd.MultiIndex.from_product([['Time','Node'],
                                     ['Cut', 'No Cut','Perfect', 'RF', 'ET', 'NN']],
                                    names=['Parameter','Setting'])

train_test_row_names = pd.MultiIndex.from_product(
    [train_test_classes, train_test_sets],
    names=['Metric', 'Set'])

train_test_df = pd.DataFrame(
    index = train_test_row_names,
    dtype = float)

train_test_df = pd.DataFrame(
    columns = train_test_cols,
    index = train_test_row_names,
    dtype = float)

In [161]:
train_test_df['Instances'] = instances

train_test_df[('Time', 'Cut')] = t_cuts
train_test_df[('Time', 'No Cut')] = t_nocuts
train_test_df[('Time', 'RF')] = t_rf
train_test_df[('Time', 'ET')] = t_et
train_test_df[('Time', 'NN')] = t_nn

train_test_df[('Time', 'Perfect')] = t_perfect

train_test_df[('Node', 'Cut')] = n_cuts
train_test_df[('Node', 'No Cut')] = n_nocuts
train_test_df[('Node', 'RF')] = n_rf
train_test_df[('Node', 'ET')] = n_et
train_test_df[('Node', 'NN')] = n_nn

train_test_df[('Node', 'Perfect')] = n_perfect

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  247.106544  143.960513  153.385419   
               Test Set   274.255724  304.319538  210.563040  256.560120   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  144.681741  144.370924  2610.230290   7071.275598   
               Test Set   275.755010  269.263330  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2591.150202  2561.117038  2561.288414   
               Test Set   5074.993594  6111.645572  6324.852839  7389.587221   

Parameter                Instances  
Setting                             
Metric         Set                  
Geometric Mean Train Set       284  
               Test Set         71

In [162]:
best_time = []
best_nodes = []

zip_time = zip(t_rf, t_et, t_nn)
zip_time = list(zip_time)

zip_nodes = zip(n_rf, n_et, n_nn)
zip_nodes = list(zip_nodes)

for i in zip_time:
    best_time.append(min(i))

for i in zip_nodes:
    best_nodes.append(min(i))

In [163]:
imp_time = []
imp_nodes = []

for i, j in zip(best_time, t_cuts):
    imp = ((i - j)/j) * 100
    imp_time.append(imp)
    
for i, j in zip(best_nodes, n_cuts):
    imp = ((i - j)/j) * 100
    imp_nodes.append(imp)

In [164]:
train_test_df[('Improvement (%)', 'Time')] = imp_time
train_test_df[('Improvement (%)', 'Nodes')] = imp_nodes

train_test_df

Parameter                       Time                                      \
Setting                          Cut      No Cut     Perfect          RF   
Metric         Set                                                         
Geometric Mean Train Set  180.289030  247.106544  143.960513  153.385419   
               Test Set   274.255724  304.319538  210.563040  256.560120   

Parameter                                                Node                \
Setting                           ET          NN          Cut        No Cut   
Metric         Set                                                            
Geometric Mean Train Set  144.681741  144.370924  2610.230290   7071.275598   
               Test Set   275.755010  269.263330  5873.772645  12660.610956   

Parameter                                                                     \
Setting                       Perfect           RF           ET           NN   
Metric         Set                                                             
Geometric Mean Train Set  2308.375949  2591.150202  2561.117038  2561.288414   
               Test Set   5074.993594  6111.645572  6324.852839  7389.587221   

Parameter                Instances Improvement (%)            
Setting                                       Time     Nodes  
Metric         Set                                            
Geometric Mean Train Set       284      -19.922514 -1.881568  
               Test Set         71       -6.452228  4.049747